# Preparation

Questo notebook contiene la pulizia preliminare del dataset e la divisione in frasi riguardanti Camera e Senato (in due csv).

In [19]:
import pandas as pd
from utils import *

df = pd.read_csv('normattiva_prep_works.csv')

df.head()

,id,title,date,num_articles,description,prep_works
0,25G00016,L. 11/2025,11/02/2025,11,Istituzione del Parco ambientale per lo svilup...,LAVORI PREPARATORI Camera dei deputati (atto n...
1,25G00014,L. 7/2025,31/01/2025,1,Conversione in legge del decreto-legge 27 dice...,LAVORI PREPARATORI Senato della Repubblica (at...
2,25G00011,L. 6/2025,31/01/2025,4,Istituzione della Giornata degli internati ita...,LAVORI PREPARATORI Camera dei deputati (atto n...
3,25G00009,L. 4/2025,24/01/2025,2,"Conversione in legge, con modificazioni, del d...",LAVORI PREPARATORI Senato della Repubblica (at...
4,25G00008,L. 2/2025,24/01/2025,35,Ratifica ed esecuzione della Convenzione tra i...,LAVORI PREPARATORI Senato della Repubblica (at...


In [20]:
# Get small dataframe [id,text] to work with
df_corpus = df[['id','prep_works']].head(50)
print(df_corpus.head())

         id                                         prep_works
0  25G00016  LAVORI PREPARATORI Camera dei deputati (atto n...
1  25G00014  LAVORI PREPARATORI Senato della Repubblica (at...
2  25G00011  LAVORI PREPARATORI Camera dei deputati (atto n...
3  25G00009  LAVORI PREPARATORI Senato della Repubblica (at...
4  25G00008  LAVORI PREPARATORI Senato della Repubblica (at...


In [21]:
# Function to remove 'LAVORI PREPARATORI ' from text
def remove_prep_works(text):
    return text.replace('LAVORI PREPARATORI ', '')

# Apply function to text
df_corpus['prep_works'] = df_corpus['prep_works'].apply(remove_prep_works)

print(df_corpus.head())

         id                                         prep_works
0  25G00016  Camera dei deputati (atto n. 400): Presentato ...
1  25G00014  Senato della Repubblica (atto n. 1335): Presen...
2  25G00011  Camera dei deputati (atto n. 1835): Presentato...
3  25G00009  Senato della Repubblica (atto n. 1315): Presen...
4  25G00008  Senato della Repubblica (atto n. 1128): Presen...


Rimpiazza le abbreviazioni

In [22]:
# Replace abbreviations

df_corpus['prep_works'] = df_corpus['prep_works'].astype(str).apply(replace_abbreviations)

In [23]:
# Per ogni riga 'id' in 'prep_works', crea una nuova colonna 'sentences' con l'array di frasi
df_corpus['sentences'] = df_corpus['prep_works'].apply(lambda text: splitter_e_tokenizzatore(text)[0])

df_corpus['sentences'].head().iloc[0]

["Camera dei deputati (atto n. 400): Presentato dall'Onorevole Marco Simiani (PD-IDP) e altri, in data 19 ottobre 2022.",
 'Assegnato alla VIII Commissione (Ambiente, territorio e lavori pubblici), in sede referente, il 23 novembre 2022, con i pareri delle commissioni I (Affari costituzionali, della Presidenza del Consiglio e Interni), II (Giustizia), V (Bilancio, tesoro e programmazione), XI (Lavoro pubblico e privato) e per le Questioni regionali.',
 'Esaminato dalla Commissione VIII (Ambiente, territorio e lavori pubblici), in sede referente, il 15, il 21 e il 28 novembre 2023; il 17 gennaio 2024; il 7 febbraio 2024; il 23 aprile 2024; il 5 agosto 2024; il 2 ottobre 2024.',
 "Esaminato in Aula il 16 ottobre 2024 e trasferito all'VIII Commissione (Ambiente, territorio e lavori pubblici), in sede legislativa, il 17 ottobre 2024.",
 'Nuovamente assegnato alla VIII Commissione (Ambiente, territorio e lavori pubblici), in sede legislativa, il 17 ottobre 2024, con i pareri delle commissio

Divisione in Camera e Senato

In [24]:
# Funzione per dividere il testo in base ai titoli "Camera dei deputati" e "Senato della Repubblica"
def divide_in_eventi(frasi):
    # Trova l'indice dove inizia la parte del Senato e Camera
    senato_start = next((i for i, frase in enumerate(frasi) if 'Senato della Repubblica' in frase), None)
    camera_start = next((i for i, frase in enumerate(frasi) if 'Camera dei deputati' in frase), None)

    if camera_start < senato_start:
        # Dividi la lista in due sezioni: camera e senato
        camera_frasi = frasi[:senato_start]
        senato_frasi = frasi[senato_start:]
    else:
        senato_frasi = frasi[:camera_start]
        camera_frasi = frasi[camera_start:]
        
    return camera_frasi, senato_frasi

# Applicare la funzione a tutte le righe e separare i risultati in due nuove colonne
df_corpus[['camera', 'senato']] = pd.DataFrame(df_corpus['sentences'].apply(divide_in_eventi).tolist(), index=df_corpus.index)

# Mostra il DataFrame
df_corpus.head()

,id,prep_works,sentences,camera,senato
0,25G00016,Camera dei deputati (atto n. 400): Presentato ...,[Camera dei deputati (atto n. 400): Presentato...,[Camera dei deputati (atto n. 400): Presentato...,[Senato della Repubblica (atto n. 1275): Asseg...
1,25G00014,Senato della Repubblica (atto n. 1335): Presen...,[Senato della Repubblica (atto n. 1335): Prese...,[Camera dei deputati (atto n. 2206): Assegnato...,[Senato della Repubblica (atto n. 1335): Prese...
2,25G00011,Camera dei deputati (atto n. 1835): Presentato...,[Camera dei deputati (atto n. 1835): Presentat...,[Camera dei deputati (atto n. 1835): Presentat...,[Senato della Repubblica (atto n. 1239): Asseg...
3,25G00009,Senato della Repubblica (atto n. 1315): Presen...,[Senato della Repubblica (atto n. 1315): Prese...,[Camera dei deputati (atto n. 2196): Assegnato...,[Senato della Repubblica (atto n. 1315): Prese...
4,25G00008,Senato della Repubblica (atto n. 1128): Presen...,[Senato della Repubblica (atto n. 1128): Prese...,[Camera dei deputati (atto n. 2031): Assegnato...,[Senato della Repubblica (atto n. 1128): Prese...


In [25]:
print(type(df_corpus['camera'].iloc[0]))

<class 'list'>


In [26]:
import ast

# Crea DataFrame solo per df['camera']
df_camera = pd.DataFrame(df_corpus, columns=['id', 'camera'])

# Salva come CSV
df_camera.to_csv('camera.csv', index=False)

# Verifica il risultato
print(df_camera.head())

         id                                             camera
0  25G00016  [Camera dei deputati (atto n. 400): Presentato...
1  25G00014  [Camera dei deputati (atto n. 2206): Assegnato...
2  25G00011  [Camera dei deputati (atto n. 1835): Presentat...
3  25G00009  [Camera dei deputati (atto n. 2196): Assegnato...
4  25G00008  [Camera dei deputati (atto n. 2031): Assegnato...


In [10]:
print(type(df_camera['camera'].iloc[0]))

<class 'list'>


In [8]:
# Crea DataFrame solo per df['camera']
df_senato = pd.DataFrame(df_corpus, columns=['id', 'senato'])

# Salva come CSV
df_senato.to_csv('senato.csv', index=False)

# Verifica il risultato
print(df_senato.head())

         id                                             senato
0  25G00016  [Senato della Repubblica (atto n. 1275): Asseg...
1  25G00014  [Senato della Repubblica (atto n. 1335): Prese...
2  25G00011  [Senato della Repubblica (atto n. 1239): Asseg...
3  25G00009  [Senato della Repubblica (atto n. 1315): Prese...
4  25G00008  [Senato della Repubblica (atto n. 1128): Prese...


Questa parte di seguito è da eliminare da questo notebook, lo lasciamo solo per mantenere l'output.

## Bigrammi

In [ ]:
print("Bigrammi più comuni:\n")

ngrams = most_common_ngrams_corpus(prep_works_texts, 2, 50)
print_ngrams(ngrams)

sede referente: 69
affari esteri: 45
affari costituzionali: 43
presidenza consiglio: 43
esaminato aula: 37
lavoro pubblico: 33
pubblico privato: 33
2024 pareri: 33
pareri commissioni: 32
lavori pubblici: 31
affari sociali: 29
esaminato commissione: 28
politiche europea: 27
questioni regionali: 26
novembre 2024: 24
bilancio tesoro: 23
tesoro programmazione: 23
dicembre 2024: 23
esteri difesa: 23
costituzionali presidenza: 22
consiglio interni: 22
programmazione economica: 22
economica bilancio: 22
regionali esaminato: 21
ottobre 2024: 21
costituzionali affari: 21
affari presidenza: 21
consiglio ordinamento: 21
ordinamento generale: 21
generale stato: 21
stato pubblica: 21
pubblica amministrazione: 21
amministrazione editoria: 21
editoria digitalizzazione: 21
2024 approvato: 21
commercio turismo: 21
lavori preparatori: 20
camera deputati: 20
deputati atto: 20
senato repubblica: 20
repubblica atto: 20
commissioni 1ª: 20
1ª affari: 20
commissioni affari: 18
v bilancio: 18
5ª programmazione

# Trigrammi

In [ ]:
print("Trigrammi più comuni:\n")

ngrams = most_common_ngrams_corpus(prep_works_texts, 3, 50)
print_ngrams(ngrams)



Trigrammi più comuni:
lavoro pubblico privato: 33
2024 pareri commissioni: 24
bilancio tesoro programmazione: 23
affari esteri difesa: 23
affari costituzionali presidenza: 22
costituzionali presidenza consiglio: 22
presidenza consiglio interni: 22
programmazione economica bilancio: 22
questioni regionali esaminato: 21
affari costituzionali affari: 21
costituzionali affari presidenza: 21
affari presidenza consiglio: 21
presidenza consiglio ordinamento: 21
consiglio ordinamento generale: 21
ordinamento generale stato: 21
generale stato pubblica: 21
stato pubblica amministrazione: 21
pubblica amministrazione editoria: 21
amministrazione editoria digitalizzazione: 21
camera deputati atto: 20
senato repubblica atto: 20
1ª affari costituzionali: 20
commissioni 1ª affari: 19
commissioni affari costituzionali: 18
v bilancio tesoro: 18
5ª programmazione economica: 18
affari sociali sanità: 18
sociali sanità lavoro: 18
sanità lavoro pubblico: 18
pubblico privato previdenza: 18
privato previden